In [1]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pickle
import csv
import re
#[newmm,icu,longest-matching,pylexto,mm,deepcut,wordcutpy]
pattern = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''")

In [2]:
f = open("./train.txt", "r")
#x_train = np.array([word_tokenize(line.strip(),'newmm',whitespaces=True) for line in f.readlines()])
x_train = []
for ss in f.readlines():
    clean_string = ''.join([char for char in ss if not char in re.findall(pattern, ss)])
    x_train.append(word_tokenize(clean_string, 'newmm',whitespaces=True)[:30])
f.close()
x_train = np.array(x_train)
f = open("./train_label.txt", "r")
y_train = pd.Series([line.strip() for line in f.readlines()])
f.close()
x_train.shape,y_train.shape

((24345,), (24345,))

In [3]:
x_train[3]

['ข้างหลัง', 'เลียนแบบ', ' ', 'mg', 'ชัด', 'ชัด']

In [4]:
label=pd.factorize(y_train)[1]
print(label)

Index(['neu', 'neg', 'pos', 'q'], dtype='object')


In [5]:
parameters = [
    #{'C': [10, 100, 1000], 'kernel': ['linear']},
    #{'C': [100], 'gamma': [0.1,0.01], 'degree':[2,3],'kernel': ['poly'],'decision_function_shape':['ovr']},
    {'C': [10], 'gamma': [0.00001], 'degree':[2], 'kernel': ['rbf'],'decision_function_shape':['ovr']}
             ]

In [6]:
f = open("./test.txt", "r")
#x_test = [word_tokenize(''.join([i for i in line.strip() if not i.isdigit()]),'newmm',whitespaces=True) for line in f.readlines()]
x_test = []
for ss in f.readlines():
    clean_string = ''.join([char for char in ss if not char in re.findall(pattern, ss)])
    x_test.append(word_tokenize(clean_string, 'newmm',whitespaces=True)[:30])
f.close()
print(x_test[9])

['รถ', 'ก็', 'มี', 'ข้อผิดพลาด', 'ด้วยกัน', 'ทุก', 'ค่าย', ' ', 'แต่', 'ดูแล', 'ลูกค้า', 'อย่างไร', ' ', 'ตรง', 'ที่', ' ', 'mazda', 'เป็นปัญหา', 'อยู่', 'เนี่ย', ' ', 'ดีเซล', ' ', 'cx', ' ', 'mazda', '  ', 'อดีต', 'ช่าง', 'พอ']


In [7]:
dict_size = 7700
vectorizer = TfidfVectorizer(max_features=dict_size,analyzer='char_wb',ngram_range=(1, 3))
X = vectorizer.fit_transform([' '.join(i) for i in x_train])
label=list(pd.factorize(y_train)[1])
Y=[label.index(i) for i in y_train]

In [8]:
svc = svm.SVC(cache_size=400)
clf = GridSearchCV(svc, parameters, cv=10,n_jobs=-1)
clf.fit(X, Y)

y_pred = clf.predict(X)

acc = accuracy_score(y_pred,Y)
print(acc,dict_size)

test_data = vectorizer.transform([' '.join(i) for i in x_test])
output = clf.predict(test_data)
y_pred=[]
for i in output:
    y_pred.append(label[i])

with open('file_'+str(dict_size)+'_try.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Id','Class'])
    [writer.writerow([i+1,j]) for i,j in enumerate(y_pred)]

pickle.dump(clf, open('SVM_'+str(dict_size)+'_try.sav', 'wb'))
#del X,Y,y_pred,output,acc,clf,svc,label,vectorizer
print('Finished!')

0.5850482645307045 7700
Finished!
